In [ ]:
import tensorflow as tf
import numpy
import utils

import keras
from keras.layers import Input, LSTM, LSTMCell
from keras.callbacks import Tensorboard

In [ ]:
n_h = 128

# Input shape (M, T, P)
X_train, Y_train, X_test, Y_test = utils.getGenerativeSequence()
M, Tx, n_values = X_train.shape
Ty = 15

In [ ]:
reshapor = Reshape((1, n_values))
LSTM_cell_layer1 = LSTM(n_h, activation='relu', return_state=True)
# LSTM_cell_layer2 = LSTM(n_h, return_state = True)
# LSTM_cell_layer3 = LSTM(n_h, return_state = True)
densor = Dense(n_values)
intermmediate_out = Dense(n_h, activation='relu')

In [ ]:
# def lstmModel3Layers(Tx, n_h , n_values):
    
#     X_inp = Input(shape=(Tx, n_values))
    
#     a10 = Input(shape=(n_h,), name='a1')
#     c10 = Input(shape=(n_h,), name='c1')
#     a1 = a10
#     c1 = c10
    
#     a20 = Input(shape=(n_h,), name='a2')
#     c20 = Input(shape=(n_h,), name='c2')
#     a2 = a20
#     c2 = c20
    
#     a30 = Input(shape=(n_h,), name='a3')
#     c30 = Input(shape=(n_h,), name='c3')
#     a3 = a30
#     c3 = c30
    
#     outputs = []
    
#     for t in range(Tx):
        
#         x = Lambda(lambda x: X[:,t,:])(X)
#         x = reshapor(x)
#         a1, _, c1 = LSTM_cell_layer1(x, initial_state=[a1, c1])
#         out_hidden1 = intermmediate_out(a1)
        
#         a2, _, c2 = LSTM_cell_layer2(out_hidden1, initial_state=[a2, c2])
#         out_hidden2 = intermmediate_out(a2)
        
#         a3, _, c3 = LSTM_cell_layer3(out_hidden2, initial_state=[a3, c3])
#         out = densor(a3)
#         outputs.append(out)
        
#     model = Model([X, a10, c10, a20, c20, a30, c30], outputs)
    
#     return model

In [ ]:
def lstmModel1LayerTrain(Tx, Ty, n_h, n_a, n_values):
    
    X_inp = Input(shape=(Tx, n_values))
    
    s0 = Input(shape=(n_h,), name='s0')
    c0 = Input(shape=(n_h,), name='c0')
    s = s0
    c = c0
    
    outputs = []
    
    a = LSTM(n_a, activation='relu')(X_inp)
    for t in range(Ty):
        s, _, c = LSTM_cell_layer1(a, initial_state=[s, c])
        out = densor(s)
        outputs.append(out)
        a = out
        
    model = Model([X, s0, c0], outputs)
    
    return model

In [ ]:
model = lstmModel1LayerTrain(Tx, Ty, n_h, n_a, n_values)

model.save_weights("model.h5")

In [ ]:
opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.01)

model.compile(optimizer=opt, loss='mean_squared_error', metrics=['accuracy'])

In [ ]:
a0 = np.zeros((M, n_h))
c0 = np.zeros((M, n_h))

In [ ]:
NUM_EPOCHS = 20
BATCH_SIZE = 2

callbacks = [
    Tensorboard(write_grads=True)
]
model.fit([X_train, a0, c0], list(Y_train), epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, callbacks=callbacks)

In [ ]:
model.evaluate(X_test, Y_test, batch_size=BATCH_SIZE)

In [ ]:
X_predict = utils.getPredictingData()

print model.predict(X_predict)